In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium


Autosaving every 180 seconds


<ipython-input-1-e4476ecac24a>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/home/cat/.conda/envs/donato/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
############################################
################# Load data ################
############################################

#turn off plotting or multiprocessing will crash
plt.switch_backend('agg')

#
def get_sessions_from_directory_listing(root_dir,
                                       animal_id):
    
    sessions = os.listdir(root_dir+animal_id)
    print ("sessions: ", sessions)
    
    return sessions
    
#
animal_ids = [
    
    'DON-002865',
    'DON-003165',
    'DON-003343',
            
    'DON-006084',
    'DON-006085',
    'DON-006087',
    

#     'DON-008497',
#     'DON-008498',
#     'DON-008499',
    
    # 'DON-009191',
    # 'DON-009192',
    # 'DON-010473',
    # 'DON-010477',
    

       
    
    
]
              
############################################
############################################
############################################
root_dir = '/media/cat/4TB/donato/'
dir_ = '/002P-F/tif/'

#
for animal_id in animal_ids:

    #
    print ("Processing: ", animal_id)

    #
    sessions = get_sessions_from_directory_listing(root_dir,
                                                  animal_id)

    #
    for session in sessions:
        #
        fname_check  =  os.path.join(root_dir, 
                                     animal_id, 
                                     session+dir_, 'suite2p','plane0',
                                     'goodcell_correlations_array_post_deduplication_filtered.npy'
                                      )

        #
        #if os.path.exists(fname_check):
        #   continue

        #
        #try: 
        try:
            #
            print ('')
            print ('')
            print ('')
            print ("processing: ", session)

            #if session != '20220302':
            #    continue

            c = calcium.Calcium()
            c.root_dir = root_dir
            c.data_dir = os.path.join(root_dir, animal_id, session+dir_, 'suite2p','plane0')
            c.animal_id = animal_id
            c.session = session
            c.detrend_model_order = 1
            c.n_cores = 26
            
            #
            c.load_suite2p()           #note: this already deletes the non-trusted suite2p cells
            c.save_python = True
            c.save_matlab = False

            # 
            c.min_width_event_onphase = 30
            c.min_width_event_upphase = 10
            c.recompute_binarization = True

            # not used currently
            c.dff_min = 0.02   #this is the final threshold required by putative spikes
            c.percentile_threshold = 0.999999
            c.use_upphase = True

            #
            c.show_plots =False
            c.save_plots = True
            c.remove_ends = False                     # delete the first and last x seconds in case [ca] imaging had issues
            c.detrend_filter_threshold = 0.001
            c.mode_window = 30*60  # None: compute mode on entire time; Value: sliding window based - baseline detection # of frames to use to compute mode

            #
            c.load_binarization()   
            c.save_sample_traces()
            c.show_rasters(save_image=True)

            #
            binarization_method = 'upphase'
            if binarization_method=='onphase':
                traces = c.F_onphase_bin
            elif binarization_method=='upphase':
                traces = c.F_upphase_bin
            else:
                print ("METHOD NOT FOUND")

            
            ###################################################################
            ########## cleanup cells + compute pairwise correlations ##########
            ###################################################################
            #c.zscore = True                     # here we compute correlations based on N shuffles of the data and zscore
            for z in [[False,False],[True, False],[False,True]]:
                ###################################################################
                c.subselect_moving_only = z[0]
                c.subselect_quiescent_only = z[0]
                #c.subselect_moving_only = False
                #c.subselect_quiescent_only = False
                c.corr_parallel_flag = True
                c.zscore = True                     # here we compute correlations based on N shuffles of the data and zscore
                                                    # otherwise we save the raw correlation also
                c.n_tests_zscore = 1000

                #
                c.correlation_datatype = 'upphase'      # filtered vs. upphase

                # 
                c.n_cores = 32
                c.recompute_correlation = False
                c.binning_window = 30        # binning window in frames
                c.subsample = 1              # subsample traces by this factor
                c.scale_by_DFF = True        # scale traces by DFF
                c.shuffle_data = False       # shuffle traces before computing correlations 
                c.compute_correlations()

                # remove duplicate neurons requires correlation array to be computed first; it uses physical overalp but also correlation threshold

                #
                c.load_footprints()
                c.deduplication_method = 'overlap'      # 'overlap'; 'centre_distance'
                c.corr_min_distance = 8                 # min distance for centre_distance method - NOT USED HERE
                c.corr_max_percent_overlap = 0.25       # max overlap for overlap method
                c.corr_threshold = 0.3                  # max correlation allowed for high overlap
                c.zscore_threshold = 3.0                # zscore threshold for high overlap

                #  
                c.corr_delete_method = 'lowest_snr' #'highest_connected', lowest_snr'
                c.recompute_deduplication = True
                c.remove_duplicate_neurons()     # this removes duplicate neurons and saves non-duplicate version of correlation array

                #
                print ("DONE")
        
        #
        except:
           print ("... errror loading : ", session, " ", animal_id)

print ("DONE...")

Processing:  DON-002865
sessions:  ['DON-002865_20210210', 'DON-002865_202102114', 'DON-002865_202102115', 'DON-002865_202102113', 'DON-002865_202102119', 'DON-002865_20210211', 'DON-002865_202102116', 'DON-002865_202102117', 'DON-002865_202102118', 'DON-002865_202102112']



processing:  DON-002865_20210210


model filter: remove bleaching or trends: 100%|██████████| 1064/1064 [00:08<00:00, 122.01it/s]
1071it [00:07, 151.41it/s]                         
binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1064/1064 [00:02<00:00, 516.25it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1064/1064 [00:15<00:00, 69.93it/s] 


...saving data...
   Queiscent periods (seconds):  87.73333333333333
   Running periods (seconds):  2757.4
... computing pairwise pearson correlation ...
 RASTERS IN:  (1064, 86186)
 BINNING WINDOW:  30


  0%|          | 0/1064 [00:00<?, ?it/s]

In [6]:
d = np.load('/media/cat/4TB/donato/DON-002865/DON-002865_20210210/002P-F/tif/suite2p/plane0/allcell_correlation_array_upphase_quiescent.npy')

print (d.shape)

c = np.load('/media/cat/4TB/donato/DON-002865/DON-002865_20210210/002P-F/tif/suite2p/plane0/good_ids_post_deduplication_upphase_quiescent.npy')
print (c.shape)

(1064, 1064, 2)
(922,)
